Basic scanner
===============
<hr>

The goal of this file is to learn how to make a basic agent using langchain and gpt. 

The agents goal is simple:
- discover all hosts on the network
- discover all open ports
- make a guess about what the device could be

In [1]:
import os
import openai
from dotenv import load_dotenv
from langchain.tools import ShellTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.agents.agent_toolkits import create_python_agent

In [2]:
load_dotenv()
openai.organization = os.getenv("OPENAI_ORGANIZATION")
openai.api_key = os.getenv("OPENAI_API_KEY")

Creating 2 different agents, one for running shell code and one for running python code.

While agents are supposed to be able to have multiple tools enabled at once, this method makes the actions of the agent a little bit more predictable. I find it works better when you already know what needs to be done.

In [3]:
shell_agent = create_python_agent(
    llm=OpenAI(temperature=0, max_tokens=1000),
    tool=ShellTool(),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_execution_time=100,
)
python_agent = create_python_agent(
    llm=OpenAI(temperature=0, max_tokens=1000),
    tool=PythonREPLTool(),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_execution_time=100,
)

In [4]:
output = shell_agent("List all devices on my network")



> Entering new AgentExecutor chain...
 I need to run a command to list all devices
Action: terminal
Action Input: nmap -sP 192.168.1.0/24

/home/antone/Documents/uni-notes.git/final-year-project/research/basic-agent/.venv/lib/python3.11/site-packages/langchain/tools/shell/tool.py:32: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(



Observation: Starting Nmap 7.94 ( https://nmap.org ) at 2023-09-27 12:04 BST
Nmap scan report for 192.168.1.1
Host is up (0.00047s latency).
Nmap scan report for 192.168.1.114
Host is up (0.0053s latency).
Nmap scan report for 192.168.1.121
Host is up (0.0027s latency).
Nmap scan report for 192.168.1.146
Host is up (0.067s latency).
Nmap scan report for 192.168.1.179
Host is up (0.00013s latency).
Nmap scan report for 192.168.1.181
Host is up (0.042s latency).
Nmap scan report for 192.168.1.182
Host is up (0.043s latency).
Nmap done: 256 IP addresses (7 hosts up) scanned in 3.76 seconds

Thought: I now know the list of devices on my network
Final Answer: 192.168.1.1, 192.168.1.114, 192.168.1.121, 192.168.1.146, 192.168.1.179, 192.168.1.181, 192.168.1.182

> Finished chain.


Generative AI always gives you a different response, its best to format it with generative AI instead of structured code.

In [5]:
ips_list = python_agent(f"Get all the IP addresses from this string as a python list: \"{output['output']}\"")



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


 I need to split the string into individual IP addresses
Action: Python_REPL
Action Input: ip_addresses = "192.168.1.1, 192.168.1.114, 192.168.1.121, 192.168.1.146, 192.168.1.179, 192.168.1.181, 192.168.1.182".split(', ')
Observation: 
Thought: I need to put the IP addresses into a list
Action: Python_REPL
Action Input: ip_list = [ip.strip() for ip in ip_addresses]
Observation: 
Thought: I now know the final answer
Final Answer: ['192.168.1.1', '192.168.1.114', '192.168.1.121', '192.168.1.146', '192.168.1.179', '192.168.1.181', '192.168.1.182']

> Finished chain.


In [6]:
import ast
devices = ast.literal_eval(ips_list['output'])
print(devices)

['192.168.1.1', '192.168.1.114', '192.168.1.121', '192.168.1.146', '192.168.1.179', '192.168.1.181', '192.168.1.182']


For each found device, we run a simple port scan to find out what ports and services are open.

In [7]:
network_scan = {}
for address in devices:
    output = shell_agent(f"List all the ports and services for this address: {address}")
    network_scan[str(address)] = output['output']




> Entering new AgentExecutor chain...
 I need to use a terminal command to list all the ports and services
Action: terminal
Action Input: nmap 192.168.1.1
Observation: Starting Nmap 7.94 ( https://nmap.org ) at 2023-09-27 12:04 BST
Nmap scan report for 192.168.1.1
Host is up (0.012s latency).
Not shown: 995 closed tcp ports (conn-refused)
PORT      STATE SERVICE
80/tcp    open  http
5500/tcp  open  hotline
8180/tcp  open  unknown
9000/tcp  open  cslistener
10004/tcp open  emcrmirccd

Nmap done: 1 IP address (1 host up) scanned in 0.23 seconds

Thought: I now know the ports and services for this address
Final Answer: The ports and services for 192.168.1.1 are 80/tcp (http), 5500/tcp (hotline), 8180/tcp (unknown), 9000/tcp (cslistener), and 10004/tcp (emcrmirccd).

> Finished chain.


> Entering new AgentExecutor chain...
 I need to use a terminal command to list all the ports and services
Action: terminal
Action Input: nmap 192.168.1.114
Observation: Starting Nmap 7.94 ( https://nmap.

In [8]:
from pprint import pprint
pprint(network_scan)

{'192.168.1.1': 'The ports and services for 192.168.1.1 are 80/tcp (http), '
                '5500/tcp (hotline), 8180/tcp (unknown), 9000/tcp '
                '(cslistener), and 10004/tcp (emcrmirccd).',
 '192.168.1.114': '53/tcp (domain), 5000/tcp (upnp), 7000/tcp '
                  '(afs3-fileserver), 7100/tcp (font-service), 49152/tcp '
                  '(unknown), 49163/tcp (unknown), 62078/tcp (iphone-sync)',
 '192.168.1.121': '49152/tcp open unknown, 49157/tcp open unknown, 62078/tcp '
                  'open iphone-sync',
 '192.168.1.146': 'All 1000 scanned ports on 192.168.1.146 are in ignored '
                  'states.',
 '192.168.1.179': 'No open ports or services were found on 192.168.1.179.',
 '192.168.1.181': '49152/tcp open unknown, 62078/tcp open iphone-sync',
 '192.168.1.182': '49153/tcp open unknown, 62078/tcp open iphone-sync'}


Using GPT3, we compile the report to make it human readable.

In [12]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003')
text= llm(f"Create a simple network report out of the information below. Don't leave out any of the ports or services. Make a guess as to what each device could be. \n\n{network_scan}", max_tokens=1000)

In [13]:
print(text)



Network Report: 

Device 192.168.1.1: This device appears to be a web server with the ports 80/tcp (http), 5500/tcp (hotline), 8180/tcp (unknown), 9000/tcp (cslistener), and 10004/tcp (emcrmirccd) open. 

Device 192.168.1.114: This device appears to be a domain server with the ports 53/tcp (domain), 5000/tcp (upnp), 7000/tcp (afs3-fileserver), 7100/tcp (font-service), 49152/tcp (unknown), 49163/tcp (unknown), 62078/tcp (iphone-sync) open. 

Device 192.168.1.121: This device appears to be a mobile device with the ports 49152/tcp open unknown, 49157/tcp open unknown, 62078/tcp open iphone-sync open. 

Device 192.168.1.146: No open ports or services were found on this device. 

Device 192.168.1.179: No open ports or services were found on this device. 

Device 192.168.1.181: This device appears to be a mobile device with the ports 49152/tcp open unknown, 62078/tcp open iphone-sync open. 

Device 192.168.1.182: This device appears to be a mobile device with the ports 49153/tcp open unkno